In [1]:
import spacy
import re
import pandas as pd

In [2]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 63.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# Load the SpaCy model (install with `python -m spacy download en_core_web_md`)
nlp = spacy.load("en_core_web_md")

In [4]:
def extract_company_names(text):
    """
    Extracts company names from a given text using SpaCy NER and regex patterns.

    Args:
        text (str): Input text containing potential company names.

    Returns:
        list: A list of extracted company names, sorted and deduplicated.
    """
    # Extract organizations using SpaCy NER
    doc = nlp(text)
    spacy_companies = [ent.text for ent in doc.ents if ent.label_ == "ORG"]

    # Define common company suffixes (expand this list as needed)
    suffixes = [
        "Inc", "Inc.", "Ltd", "Ltd.", "LLC", "Corp", "Corp.", "Corporation",
        "GmbH", "S.A.", "Co", "Co.", "PLC", "LLP", "Pvt. Ltd.", "Limited",
        "AG", "S.A.S.", "NV", "AB", "AS", "SA", "Bhd", "Plc", "L.P.", "Group",
        "Holdings", "Enterprises", "Solutions", "Technologies", "International"
    ]

    # Sort suffixes by descending length to prioritize longer matches
    suffixes_sorted = sorted(suffixes, key=lambda s: len(s), reverse=True)
    suffix_pattern = '|'.join(map(re.escape, suffixes_sorted))

    # Regex pattern to match company names with suffixes (case-insensitive suffixes)
    # Allows apostrophes and hyphens in company names (e.g., "O'Neil Consulting Inc.")
    company_pattern = rf'''
        \b                      # Word boundary
        (                       # Company name part (captured)
            [A-Z]               # Starts with uppercase letter
            [a-zA-Z\'\-]+       # Followed by letters, apostrophes, or hyphens
            (?:\s+              # One or more whitespace
            [A-Z]               # Next word starts with uppercase
            [a-zA-Z\'\-]+       # Followed by letters, apostrophes, or hyphens
            )*                  # Repeat for multi-word names
        )
        \s+                     # Whitespace before suffix
        ((?i:{suffix_pattern})) # Case-insensitive suffix (captured)
        \b                      # Word boundary
    '''

    # Find all regex matches
    regex_companies = []
    for match in re.finditer(company_pattern, text, flags=re.VERBOSE):
        name_part = match.group(1)
        suffix = match.group(2)
        full_name = f"{name_part} {suffix}"
        regex_companies.append(full_name)

    # Combine results and deduplicate
    combined = list(set(spacy_companies + regex_companies))
    combined.sort()
    return combined

In [5]:
# Example usage
if __name__ == "__main__":
    sample_text = """
    Apple Inc. announced a partnership with XYZ Corporation. Tesla and SpaceX,
    based in California, are expanding. A startup called HealthTech Pvt. Ltd.
    joined the market. Google's parent, Alphabet Inc., also participated.
    Siemens AG and JPMorgan Chase & Co. were mentioned.
    """
    companies = extract_company_names(sample_text)
    print("Extracted Company Names:")
    for name in companies:
        print(f"- {name}")

Extracted Company Names:
- Alphabet Inc
- Alphabet Inc.
- Apple Inc
- Google
- HealthTech Pvt. Ltd.
    
- JPMorgan Chase & Co.
- Siemens AG
- SpaceX
- Tesla
- XYZ Corporation


In [6]:
#extract_company_names(news_list[500])

In [7]:
import json

In [8]:
def read_json_file(file_path):
    """
    Reads a JSON file and returns its content as a Python object.

    Args:
        file_path (str): The path to the JSON file.

    Returns:
        dict or list: The content of the JSON file as a Python dictionary or list,
                     or None if an error occurs.
    """
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in '{file_path}'")
        return None
    except Exception as e:
         print(f"An unexpected error occurred: {e}")
         return None

In [9]:
file_path = '/content/2023_processed.json'
data_2023 = read_json_file(file_path)

In [10]:
#data_2023_df = pd.DataFrame(data_2023)

In [11]:
#data_2023_df.shape

In [ ]:
# https://huggingface.co/datasets/luckycat37/financial-news-dataset

In [12]:
file_path = '/content/2022_processed.json'
data_2022 = read_json_file(file_path)
#data_2022_df = pd.DataFrame(data_2022)

In [13]:
file_path = '/content/2021_processed.json'
data_2021 = read_json_file(file_path)
#data_2021_df = pd.DataFrame(data_2021)

In [14]:
file_path = '/content/2020_processed.json'
data_2020 = read_json_file(file_path)
#data_2020_df = pd.DataFrame(data_2020)

In [15]:
file_path = '/content/2019_processed.json'
data_2019 = read_json_file(file_path)
#data_2019_df = pd.DataFrame(data_2019)

In [16]:
file_path = '/content/2018_processed.json'
data_2018 = read_json_file(file_path)
#data_2018_df = pd.DataFrame(data_2018)

In [17]:
file_path = '/content/2017_processed.json'
data_2017 = read_json_file(file_path)
#data_2017_df = pd.DataFrame(data_2017)

In [18]:
#data_2023_df.columns

In [19]:
def json_to_dataframe_selected_keys(json_data, keys_to_extract):
    """
    Converts a JSON string or a list of JSON-like dictionaries to a Pandas DataFrame,
    selecting only the specified keys.

    Args:
        json_data (str or list): A JSON string or a list of dictionaries.
                                   Each dictionary in the list (or the dictionary
                                   parsed from the JSON string) is expected to have
                                   the keys you want to extract.
        keys_to_extract (list): A list of keys whose values you want to include
                                in the DataFrame.

    Returns:
        pandas.DataFrame: A DataFrame containing the values for the selected keys.
                          Returns an empty DataFrame if the input is invalid or
                          no data is processed.
    """
    try:
        if isinstance(json_data, str):
            data = json.loads(json_data)
            if isinstance(data, dict):
                data = [data]  # Handle single JSON object
        elif isinstance(json_data, list):
            data = json_data
        else:
            print("Error: Input 'json_data' must be a JSON string or a list of dictionaries.")
            return pd.DataFrame()

        extracted_data = []
        for item in data:
            row = {}
            for key in keys_to_extract:
                if key in item:
                    row[key] = item[key]
                else:
                    row[key] = None  # Or you could choose to skip missing keys
            extracted_data.append(row)

        df = pd.DataFrame(extracted_data)
        return df

    except json.JSONDecodeError:
        print("Error: Invalid JSON string provided.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

In [20]:
def expand_json_column(df, json_column):
    """
    Separates a column containing JSON dictionaries into new key-wise columns.

    Args:
        df (pd.DataFrame): The input DataFrame.
        json_column (str): The name of the column containing JSON dictionaries.

    Returns:
        pd.DataFrame: A new DataFrame with the JSON data expanded into columns.
    """
    # Apply json.loads to convert stringified JSON to dictionaries, handling potential errors
    try:
        df[json_column] = df[json_column].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in column '{json_column}': {e}")
        return df  # Return the original DataFrame if there's a decoding error

    # Handle cases where the column might contain non-dictionary values or NaNs
    expanded_data = []
    for index, row in df.iterrows():
        json_data = row[json_column]
        if isinstance(json_data, dict):
            expanded_data.append(json_data)
        else:
            expanded_data.append({})  # Append an empty dictionary for non-dictionary values

    # Create a new DataFrame from the list of dictionaries
    expanded_df = pd.DataFrame(expanded_data, index=df.index)

    # Concatenate the new columns with the original DataFrame (excluding the JSON column)
    df_expanded = pd.concat([df.drop(columns=[json_column]), expanded_df], axis=1)

    return df_expanded

In [21]:
def json_to_df_org_entities(json_data):
    keys = ['date_publish', 'maintext', 'mentioned_companies', 'named_entities']
    df_orgen = json_to_dataframe_selected_keys(json_data, keys)
    df_orgen['org_entities'] = [[] for _ in range(len(df_orgen))]  # Create a list of empty lists with the same length as the DataFrame

    # Extract 'named_entities' if present; if 'named_entities' in data_item; Extend keys with 'word' from ORG entities
    for i, data_item in enumerate(df_orgen['named_entities']):
        df_orgen.loc[i, 'org_entities'].extend([entity for entity in data_item if entity['entity_group'] == 'ORG'])

    df_orgen.drop(columns=['named_entities'], inplace=True)

    df_orgen = df_orgen.explode('org_entities')

    df_orgene = expand_json_column(df_orgen, 'org_entities')

    df_orgene.drop(columns=['entity_group', 'normalized'], inplace=True)

    return df_orgene

In [22]:
df_2023e = json_to_df_org_entities(data_2023)

In [23]:
df_2023e.shape

(75171, 8)

In [24]:
df_2023e.head(10)

,date_publish,maintext,mentioned_companies,word,company_key,start,end,score
0,2023-06-23 05:38:00,"LOS GATOS, Calif. (AP) — At Tyler Malek's ice ...",[C],C,C,754,755,0.751716
1,2023-08-26 14:00:17,"The worst result, after buying shares in a com...",[T],T,T,1633,1634,0.965930
2,2023-12-06 16:57:28,(Bloomberg) -- An insolvency filing by Signa H...,[BA],Ba,BA,1864,1866,0.866735
3,2023-06-14 07:21:56,By John Revill\nZURICH (Reuters) - Swiss citiz...,[GOOGL],Google,GOOGL,1015,1021,0.995022
4,2023-01-10 20:23:00,"WAXHAW, N.C., Jan. 10, 2023 /PRNewswire/ -- Na...",[MA],Ma,MA,773,775,0.682062
4,2023-01-10 20:23:00,"WAXHAW, N.C., Jan. 10, 2023 /PRNewswire/ -- Na...",[MA],Ma,MA,1128,1130,0.815940
5,2023-09-11 02:03:41,By Scott Murdoch\nSYDNEY (Reuters) - Asia stoc...,"[BABA, WFC]",Alibaba Group,BABA,527,540,0.999194
5,2023-09-11 02:03:41,By Scott Murdoch\nSYDNEY (Reuters) - Asia stoc...,"[BABA, WFC]",Wells Fargo,WFC,873,884,0.970640
6,2023-09-12 04:07:10,US tech stocks surged on Monday after Tesla re...,[TSLA],Tesla,TSLA,38,43,0.994540
6,2023-09-12 04:07:10,US tech stocks surged on Monday after Tesla re...,[TSLA],Tesla,TSLA,92,97,0.994072


In [26]:
df_2023e[df_2023e['mentioned_companies'].apply(len) >1].iloc[6015:6030]

,date_publish,maintext,mentioned_companies,word,company_key,start,end,score
4231,2023-06-29 15:00:29,Microsoft (MSFT) is continuing to pile generat...,"[MSFT, GOOGL, AMZN]",Microsoft,MSFT,588,597,0.996445
4231,2023-06-29 15:00:29,Microsoft (MSFT) is continuing to pile generat...,"[MSFT, GOOGL, AMZN]",Google,GOOGL,1121,1127,0.988628
4231,2023-06-29 15:00:29,Microsoft (MSFT) is continuing to pile generat...,"[MSFT, GOOGL, AMZN]",Microsoft,MSFT,1144,1153,0.995706
4231,2023-06-29 15:00:29,Microsoft (MSFT) is continuing to pile generat...,"[MSFT, GOOGL, AMZN]",Microsoft,MSFT,1272,1281,0.996211
4231,2023-06-29 15:00:29,Microsoft (MSFT) is continuing to pile generat...,"[MSFT, GOOGL, AMZN]",Microsoft,MSFT,1473,1482,0.997328
4231,2023-06-29 15:00:29,Microsoft (MSFT) is continuing to pile generat...,"[MSFT, GOOGL, AMZN]",Microsoft,MSFT,1649,1658,0.997093
4231,2023-06-29 15:00:29,Microsoft (MSFT) is continuing to pile generat...,"[MSFT, GOOGL, AMZN]",Microsoft,MSFT,1902,1911,0.973336
4231,2023-06-29 15:00:29,Microsoft (MSFT) is continuing to pile generat...,"[MSFT, GOOGL, AMZN]",Amazon,AMZN,2065,2071,0.868976
4231,2023-06-29 15:00:29,Microsoft (MSFT) is continuing to pile generat...,"[MSFT, GOOGL, AMZN]",Microsoft,MSFT,2224,2233,0.979509
4231,2023-06-29 15:00:29,Microsoft (MSFT) is continuing to pile generat...,"[MSFT, GOOGL, AMZN]",Google,GOOGL,2304,2310,0.972302


In [ ]:
# Set the option to display the full width of columns
pd.set_option('display.max_colwidth', 200)

In [27]:
df_2022e = json_to_df_org_entities(data_2022)

In [29]:
df_2022e.shape

(59013, 8)

In [28]:
df_2021e = json_to_df_org_entities(data_2021)

In [30]:
df_2021e.shape

(57688, 8)

In [31]:
df_2020e = json_to_df_org_entities(data_2020)

In [32]:
df_2020e.shape

(6327, 8)

In [33]:
df_2019e = json_to_df_org_entities(data_2019)

In [34]:
df_2019e.shape

(5986, 8)

In [35]:
df_2018e = json_to_df_org_entities(data_2018)

In [36]:
df_2018e.shape

(1850, 8)

In [37]:
df_2017e = json_to_df_org_entities(data_2017)

In [39]:
df_2017e.shape

(1612, 8)

In [94]:
# Merge DataFrames vertically using concat
df = pd.concat([df_2017e,df_2018e], ignore_index=True, sort=False)

In [95]:
df.shape

(3462, 8)

In [96]:
# Merge DataFrames vertically using concat
df = pd.concat([df,df_2019e], ignore_index=True, sort=False)

In [97]:
# Merge DataFrames vertically using concat
df = pd.concat([df,df_2020e], ignore_index=True, sort=False)

In [98]:
# Merge DataFrames vertically using concat
df = pd.concat([df,df_2021e], ignore_index=True, sort=False)

In [99]:
# Merge DataFrames vertically using concat
df = pd.concat([df,df_2022e], ignore_index=True, sort=False)

In [100]:
# Merge DataFrames vertically using concat
df = pd.concat([df,df_2023e], ignore_index=True, sort=False)

In [101]:
df.shape

(207647, 8)

In [107]:
df.head()

,date_publish,maintext,mentioned_companies,word,company_key,start,end,score,text_len,mentioned_companies_count
0,2017-05-01 14:04:36,Yahoo Finance has obtained photos of Fitbit’s ...,[AAPL],Apple,AAPL,433,438,0.997671,3162,1
1,2017-05-01 14:04:36,Yahoo Finance has obtained photos of Fitbit’s ...,[AAPL],Apple,AAPL,1709,1714,0.996519,3162,1
2,2017-05-09 08:50:32,The pitfalls of the ‘hurry up’ hormone\nShutte...,[MA],MA,MA,113,115,0.501702,4716,1
3,2017-02-07 23:16:13,President Trump’s new head of the Federal Comm...,"[T, VZ]",T,T,1465,1466,0.976226,4007,2
4,2017-02-07 23:16:13,President Trump’s new head of the Federal Comm...,"[T, VZ]",Verizon,VZ,1472,1479,0.976185,4007,2


In [104]:
df['text_len'] = df['maintext'].apply(len)

In [106]:
df['mentioned_companies_count'] = df['mentioned_companies'].apply(len)

In [91]:
df['mentioned_companies'] = df['mentioned_companies'].apply(tuple)

In [92]:
df.head()

,date_publish,maintext,mentioned_companies,word,company_key,start,end,score
0,2017-05-01 14:04:36,Yahoo Finance has obtained photos of Fitbit’s ...,"(AAPL,)",Apple,AAPL,433,438,0.997671
1,2017-05-01 14:04:36,Yahoo Finance has obtained photos of Fitbit’s ...,"(AAPL,)",Apple,AAPL,1709,1714,0.996519
2,2017-05-09 08:50:32,The pitfalls of the ‘hurry up’ hormone\nShutte...,"(MA,)",MA,MA,113,115,0.501702
3,2017-02-07 23:16:13,President Trump’s new head of the Federal Comm...,"(T, VZ)",T,T,1465,1466,0.976226
4,2017-02-07 23:16:13,President Trump’s new head of the Federal Comm...,"(T, VZ)",Verizon,VZ,1472,1479,0.976185


In [111]:
df.sample(20)

,date_publish,maintext,mentioned_companies,word,company_key,start,end,score,text_len,mentioned_companies_count
193540,2023-10-16 11:17:45,A Comprehensive Look at Mfs Investment Grade M...,[C],C,C,649,650,0.708675,4819,1
51964,2021-05-28 00:51:00,"Sheridan, WY, May 27, 2021 (GLOBE NEWSWIRE) --...",[GS],Goldman Sachs,GS,803,816,0.999052,8071,1
123092,2022-01-23 05:25:00,GOBankingRates 6 Walmart Deals That Are Better...,[WMT],Walmart,WMT,1408,1415,0.994534,1773,1
188135,2023-03-29 22:15:00,NOT FOR DISTRIBUTION TO U.S. NEWS WIRE SERVICE...,[T],T,T,1336,1337,0.996010,11302,1
189735,2023-05-02 07:30:00,Dr Monika Ermann appointed Vice President Drug...,[C],C,C,1425,1426,0.995300,6190,1
143740,2023-07-12 00:01:00,— Recommendations are independently chosen by ...,"[AAPL, AMZN, META, T]",Facebook,META,1546,1554,0.925005,1729,4
141671,2023-06-22 03:40:40,By Makiko Yamazaki TOKYO (Reuters) -Shareholde...,[MA],Ma,MA,2232,2234,0.404058,2301,1
144294,2023-07-11 21:17:59,By Mike Scarcella (Reuters) - The U.S. Federal...,[MSFT],Microsoft,MSFT,1502,1511,0.998175,2656,1
162044,2023-08-21 14:30:01,Costco is the place to go for anyone who wants...,[COST],Costco,COST,0,6,0.872541,5055,1
56133,2021-09-17 04:03:00,Fraunhofer ISE study Ι : Vertex 670W's LCOE re...,[V],V,V,25,26,0.975898,3214,1


In [109]:
df['maintext'] = df['maintext'].str.replace('\n', ' ')
df['word'] = df['word'].str.replace('\n', ' ')
df.fillna('', inplace=True)

In [113]:
# Write DataFrame to CSV
df.to_csv('SP500_news_2017-23_with_company_entities.csv', index=False)